In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import oom
from src.oom import DiscreteValuedOOM
from src.oom.observable import ObsSequence

sys.modules['oom'] = oom

In [2]:
import pickle
import os

import numpy as np

In [3]:
with open(os.path.abspath('../data/learned_5_50.pickle'), 'rb') as picklefile:
    myoom = pickle.load(picklefile)

## Generation

In [4]:
try:
    gen = myoom.generate(1000)
    seq = ObsSequence(gen[1])
    print(seq[:10])
except ValueError:
    print("Values do not sum to 1.")

Values do not sum to 1.


## Validating Observable Operator Models

By "validating", I mean both checking the conditions a Stochastic System (SS) needs to fulfill in order to be an OOM of a valid stochastic process, as defined in [1]. By taking the system function definition and writing out its explicit definition in terms of the SS tuple's linear algebraic items, I propose two independent and sequential normalization steps, for the starting state and the operator matrices. The terms of the normalization are not necessarily these, and it is still debatable which term should not be affected by a normalization procedure.

### 1. Start state: <span style="color: #DDBBFF"> $\displaystyle f(\varepsilon) = 1 \Longleftrightarrow \sigma \omega_\varepsilon = 1 $ </span>

We can normalize the starting state vector $\displaystyle \omega_\varepsilon $ to achieve this result by $\displaystyle \omega_\varepsilon \leftarrow \omega_\varepsilon \frac{1}{\sigma \omega_\varepsilon}$.

##### a. Check actual values

In [5]:
print(myoom.lin_func * myoom.start_state)

[[0.99998611]]


##### b. Apply normalization

In [6]:
err = myoom.lin_func * myoom.start_state
myoom.start_state /= err

##### c. Check updated values

In [7]:
print(myoom.lin_func * myoom.start_state)

[[1.]]


### 2. Future distribution: <span style="color: #DDBBFF"> $\displaystyle \forall\; \bar{x} \in \Sigma^*,\; f(\bar{x}) = \sum_{x \in \Sigma} f(\bar{x} x) \Longleftrightarrow \sigma \tau_\bar{x} \omega_\varepsilon = \sigma \left(\sum_{x \in \Sigma} \tau_x \right) \tau_\bar{x} \omega_\varepsilon $ </span>

Since we want the previous relation to hold with $\displaystyle \tau_\bar{x} $ for all $\displaystyle \bar{x} \in \Sigma^* $, this amounts to having $\displaystyle \sigma = \sigma \left(\sum_{x \in \Sigma} \tau_x \right) $.

We can normalize the operator matrices $\displaystyle \tau_x $ to achieve this result by $\displaystyle [\tau_x]_j \leftarrow [\tau_x]_j \frac{[\sigma]_j}{\sigma [\mu]_j}$, where $\displaystyle \mu \equiv \sum_{x \in \Sigma} \tau_x $ and the notation $\displaystyle [M]_j$ denotes the $j$-th column of the matrix $M$.

##### a. Check actual values

In [8]:
print(myoom.lin_func * sum([op.mat for op in myoom.operators]))
print(myoom.lin_func)

[[ 1.99474191  0.1339109  -0.02175702  0.04968667  0.02613801]]
[[ 1.99482254  0.14450773 -0.00714955  0.01210436  0.01103387]]


##### b. Apply normalization

In [9]:
mu = sum([op.mat for op in myoom.operators])

for op in myoom.operators:
    lin_func_err = myoom.lin_func * mu
    
    for col in range(op.mat.shape[1]):
        op.mat[:, col] = op.mat[:, col] * myoom.lin_func[0, col] / lin_func_err[0, col]

##### c. Check updated values

In [10]:
print(myoom.lin_func * sum([op.mat for op in myoom.operators]))
print(myoom.lin_func)

[[ 1.99482254  0.14450773 -0.00714955  0.01210436  0.01103387]]
[[ 1.99482254  0.14450773 -0.00714955  0.01210436  0.01103387]]


# Generation (again)

In [11]:
try:
    gen = myoom.generate(1000)
    seq = ObsSequence(gen[1])
    print(seq[:10])
except ValueError:
    print("Values do not sum to 1.")

['Oe', 'Oa', 'Oa', 'Oc', 'Ob', 'Ob', 'Oe', 'Ob', 'Od', 'Oa']
